<a href="https://colab.research.google.com/github/bhattacharyyad/Debnath_Code/blob/master/FAISS_Exmp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
!uv pip install faiss-cpu
!uv pip install sentence-transformers

Using Python 3.12.12 environment at: /usr
Audited 1 package in 78ms
Using Python 3.12.12 environment at: /usr
Audited 1 package in 87ms


In [60]:
import pandas as pd
data = [['Where are your headquarters located?', 'location'],
['Throw my cellphone in the water', 'random'],
['Network Access Control?', 'networking'],
['Address', 'location']]
df = pd.DataFrame(data, columns = ['text', 'category'])

In [61]:
from sentence_transformers import SentenceTransformer
text = df['text']
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
vectors = encoder.encode(text)

In [62]:
import faiss

vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

In [63]:
import numpy as np

search_text = 'where is your office?'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

In [64]:
k = index.ntotal
distances, ann = index.search(_vector, k=k)

In [65]:
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
results

,distances,ann
0,0.584873,0
1,1.175950,3
2,1.644265,2
3,1.919767,1


In [66]:
df

,text,category
0,Where are your headquarters located?,location
1,Throw my cellphone in the water,random
2,Network Access Control?,networking
3,Address,location


In [67]:
merge = pd.merge(results, df, left_on='ann', right_index=True)

In [68]:
merge

,distances,ann,text,category
0,0.584873,0,Where are your headquarters located?,location
1,1.175950,3,Address,location
2,1.644265,2,Network Access Control?,networking
3,1.919767,1,Throw my cellphone in the water,random


In [69]:
labels  = df['category']
category = labels[ann[0][0]]

In [70]:
labels

,category
0,location
1,random
2,networking
3,location


In [71]:
category

'location'